# DSCI 100 Group Project Proposal

team: Rachel Liang, Ziyan He, Yuchen Zhang, Zohane Bal

In [ ]:
### Run this cell before continuing. 
library(tidyverse)
library(testthat)
library(digest)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)
options(repr.matrix.max.rows = 6)
source("tests.R")
source("cleanup.R")

### Introduction    

The dataset we are working on is downloaded from the UCI Machine Learning Repository. Although the dataset contains four databases about the heart disease diagnosis from 4 different locations, we only picked the database collected from the Cleveland Clinic Foundation since it is the only data base that has full 14 attributes recored for each heart disease diagnosis. The total instances in this database is 303.

### Load Data and Wrangle Data
This dataset does not need to be cleaned since it does not have any missing data or missing attributes. <br>
We added column names to the dataset and selected columns we needed for analysis. Also as we are treating some attributes's numeric values as categorical variables, we used as.factor() for conversion.

In [ ]:
set.seed(1234)

heart_disease_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"


heart_disease_data <- read_csv(heart_disease_url, col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs",
                                                               "restecg", "thalach", "exang", "oldpeak", "slope",
                                                               "ca", "thal", "num"))
heart_disease_data

### 14 Attributes Description
* Age: age in years
* Sex: ( 1 = male, 0 = female)
* Cp: chest pain type
Value 1: typical angina
Value 2: atypical angina
Value 3: non-anginal pain
Value 4 asymptomatic
* Trestbps: Resting Blood pressure ( in mm Hg on admission to hospital)
* Chol: Serum cholesterol in mg/dl
* Fbs: Fastig blood suger > 120 (1 = true, 0 = false)
* Dm: 1 = history if diabetes , 0 = no such history)
* Restecg: Resting electro cardiac results
Value 0 : normal
Value 1: having ST-T wave abnormality ( T wave inversion and/or ST elevation or depression of > 0.05 mV)
Value 2” Showing probable or definite left ventricular hypertrophy by Estes’ criteria
* Thalach: maximum heart rate achieved
* Exang: exercise induced angina (1 = yes ,0 = no)
* Oldpeak:ST depression induced by exercise relative to rest
* Slope: The slope of the peak exercise ST segment
Value 1: upsloping
Value 2: flat
Value 3: downsloping
* Ca: Number of major vessels (0-3) coloured by fluorosopy
* Thal: 3 = normal , 6= fixed defect , 7 = reversible defect
* Num: diagnosis of heart disease
Value 0 <50% diameter narrowing
Value 1 > 50% Diameter narrowing

In [ ]:
correlation_plot <- heart_disease_data %>% 
                select(-ca, -thal) %>% 
                ggpairs(heart_disease_data[1:12]) +
                theme(text = element_text(size = 12))


correlation_plot

In [ ]:
cp_vs_num <- heart_disease_data %>% 
            select(num, cp) %>% 
            group_by(num, cp) %>% 
            summarize(n = n()) %>%
            ggplot(aes(x = num, y = n, fill = cp)) +
            geom_bar(stat = "identity",position = "fill")
cp_vs_num

In [ ]:
thalach_vs_num <- heart_disease_data %>% 
            select(num, thalach) %>% 
            group_by(num, thalach) %>% 
            summarize(n = n()) %>%
            ggplot(aes(x = num, y = n, fill = thalach)) +
            geom_bar(stat = "identity",position = "fill")
thalach_vs_num

In [ ]:
set.seed(1234)
heart_disease_data <- read_csv(heart_disease_url, col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs",
                                                               "restecg", "thalach", "exang", "oldpeak", "slope",
                                                               "ca", "thal", "num")) %>% 
                        mutate(num = as.factor(num)) %>% 
#                         mutate(cp = as.factor(cp), fbs = as.factor(fbs), restecg = as.factor(restecg), 
#                             exang = as.factor(exang), slope = as.factor(slope), sex = as.factor(sex)) %>% 
                        select(age, sex, cp, thalach, exang, slope, num, oldpeak) 
heart_disease_data

### Check for balancing

In [ ]:
num_obs <- nrow(heart_disease_data)
        heart_disease_data %>% 
        group_by(num) %>% 
        summarize(
        count = n(),
        percentage = n() / num_obs * 100)

drawing multiple histograms and the x-axis is the num (possibility of getting heart disease : 0, 1, 2, 3) and the y-axis is the count. color the bar using position "filled" with each column (such as cp, fbs according to the larger correlation in the ggpair). if this variable has a high correlation related to getting heart disease, the proportion of color will be filled more in the larger x-axis, and otherwise. Also use medical research to add ontop of the correlation because correlation does not mean causation.

In [ ]:
set.seed(1234)
heart_disease_data_2 <- read_csv(heart_disease_url, col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs",
                                                               "restecg", "thalach", "exang", "oldpeak", "slope",
                                                               "ca", "thal", "num")) %>% 
#                         mutate(num = as.factor(num)) %>% 
#                         mutate(cp = as.numeric(as.factor(cp)), fbs = as.factor(fbs), restecg = as.factor(restecg), 
#                             exang = as.numeric(as.factor(exang)), slope = as.numeric(as.factor(slope)), 
#                                sex = as.numeric(as.factor(sex))) %>% 
                        select(cp, thalach, exang, slope, oldpeak, num) %>% 
                        mutate(num = as.factor(num))
heart_disease_data_2

In [ ]:
heart_disease_data_3 <- read_csv(heart_disease_url, col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs",
                                                               "restecg", "thalach", "exang", "oldpeak", "slope",
                                                               "ca", "thal", "num")) %>% 
                    mutate(num = replace(num, num =='2',1)) %>% 
                    mutate(num = replace(num, num =='3',1)) %>% 
                    mutate(num = replace(num, num =='4',1)) %>% 
                    select(cp, thalach, exang, slope, oldpeak, num) %>% 
                    mutate(num = as.factor(num))
                    
heart_disease_data_3                

### Check for balancing new

In [ ]:
num_obs_new <- nrow(heart_disease_data_3)
        heart_disease_data_3 %>% 
        group_by(num) %>% 
        summarize(
        count = n(),
        percentage = n() / num_obs_new * 100)

### Split Into Training and Testing Dataset
Based on the number of instances in this dataset, we decided that setting 75% of the data in the dataset as training data and the remaining 25% as testing data will make a good balance for training the classifier and evaluating the classifier's accuracy.

In [ ]:
set.seed(1234)
heart_disease_split <- initial_split(heart_disease_data_2, prop = 0.75, strata = num)
heart_disease_train <- training(heart_disease_split)
heart_disease_test <- testing(heart_disease_split)

head(heart_disease_data_2, 10)

### Summary

In [ ]:
summary(heart_disease_train)

### Visualization


The boxplot below shows the distribution of age for each different probability level of having a heart disease.

In [ ]:
# age_vs_heart_disease <- heart_disease_train %>% 
#                     ggplot(aes(x = num, y = age)) +
#                     geom_boxplot(aes(group = num)) +
#                     labs(x = "Probability of having heart disease (from 0 to 4)", y = "Patient Age") +
#                     ggtitle("Age vs. Probability of having heart disease (from 0 to 4)") +
#                     theme(text = element_text(size = 20))
                    
# age_vs_heart_disease

The graph2 below shows the distribution of different chest pain types for different probability levels of having heart disease.<br>
Chest pain type 1-4 means:
* Value 1: typical angina
* Value 2: atypical angina
* Value 3: non-anginal pain
* Value 4: asymptomatic
<br>


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10) 

sex_vs_heart_disease <- heart_disease_train %>% 
                        select(num, cp) %>% 
                        group_by(num, cp) %>% 
                        summarize(n = n()) %>% 
                        ggplot(aes(x = num, y = n, fill = cp)) +
                        geom_bar(stat = "identity",position = "fill") +
                        labs(x =  "Probability of having heart disease (from 0 to 4)", y = "distribution of different types of chest pain", fill = "chest pain type") +
                        ggtitle("Chest pain vs.  Probability of having heart disease (from 0 to 4)") +
                        theme(text = element_text(size = 18))
sex_vs_heart_disease

### Method

In order to make the prediction, there are three steps to do:
1. Create a k-nn model specification and a recipe for the fit method.
2. Train the model with the training dataset to build the classifier.
3. Use the classifier to predict the labels in the test sets.       

#### 6 Predictors chosen:
* age, sex, chest pain types (cp), thalach, exang, oldpeak, the slope of the peak exercise ST segment(slope)




### Expected Outcomes and Significance    

1. What do you expect to find?   

    Our training dataset will allow us to predict the probability of an individual contracting heart disease given patient's . 
    Additionally, we hope to discover which factors are most associated with a person's risk of developing a heart disease.  

2. What impact could such findings have?    

    With this finding, doctors would be able to analyze patients’ records in a more informative manner. It will be easier to make a swift decision to conduct further diagnostic cardiology on the patient if a certain index is abnormal as indicated in the report later, and this will significantly reduce the chance of misdiagnosis and provide more time for patient diagnosis and treatment.  

3. What future questions could this lead to?   

    As this dataset only has 303 instances which is not sufficient for making accurate predictions, we could increase the accuracy of prediction by including more instances . Furthermore, since this data set only contains patients’ data in Cleveland, it would also be interesting to observe if the key factors leading to heart disease differ between different locations if we have a heart disease data set from a collection of locations. 


### Create models

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
            set_engine("kknn")%>%
            set_mode("classification") 
knn_spec

### Create Recipe--MISSING UPSAMPLE

In [ ]:
heart_recipe <- recipe(num ~ ., data = heart_disease_train)%>%
        step_scale(all_predictors())%>%
        step_center(all_predictors()) 

# MISSING STEP_UPSAMPLE
# bake <- bake(heart_recipe,heart_disease_train)



heart_disease_vfold <- vfold_cv(heart_disease_train, v = 5, strata = num)

# FITTING Model

k_lots <- tibble(neighbors = seq(from = 1, to = 50, by = 1))

knn_fit <- workflow() %>% 
            add_recipe(heart_recipe) %>% 
            add_model(knn_spec) %>% 
            tune_grid(resamples = heart_disease_vfold, grid = k_lots)



In [ ]:
# set metrics

heart_disease_metrics <- knn_fit %>% 
                        collect_metrics()

accuracies <- heart_disease_metrics %>% 
        filter(.metric == "accuracy") 
accuracies
    
accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate") +
  theme(text = element_text(size = 12))

accuracy_vs_k

accuracies_final <- heart_disease_metrics %>% 
        filter(.metric == "accuracy") %>% 
        arrange(mean) %>% 
        tail(1)
accuracies_final


In [ ]:
# knn_tune

knn_spec_2 <- nearest_neighbor(weight_func = "rectangular", neighbors = 25) %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

knn_fit_2 <- workflow() %>% 
  add_recipe(heart_recipe) %>% 
  add_model(knn_spec_2) %>% 
  fit(data = heart_disease_train)



In [ ]:
# make predictions

heart_prediction <- predict(knn_fit_2, heart_disease_test) %>% 
                    bind_cols(heart_disease_test)

heart_metrics <- heart_prediction %>% 
                metrics(truth = num, estimate = .pred_class) %>% 
                filter(.metric == "accuracy")

mnist_conf_mat <- heart_prediction %>% 
                 conf_mat(truth = num, estimate = .pred_class)

# mnist_metrics <- mnist_predictions %>%
#         metrics(truth = y, estimate = .pred_class) %>% 
#         filter(.metric == "accuracy")
# mnist_metrics
# mnist_conf_mat <- mnist_predictions %>% 
#                 conf_mat(truth = y, estimate = .pred_class)
heart_metrics
mnist_conf_mat

## optimizing our classifier


In [ ]:
heart_recipe_new <- recipe(num ~ sex + cp + thalach + exang + slope, data = heart_disease_train)%>%
        step_scale(all_predictors())%>%
        step_center(all_predictors()) 

heart_disease_vfold <- vfold_cv(heart_disease_train, v = 5, strata = num)


# FITTING Model

k_lots <- tibble(neighbors = seq(from = 1, to = 50, by = 1))

knn_fit_new <- workflow() %>% 
            add_recipe(heart_recipe_new) %>% 
            add_model(knn_spec) %>% 
            tune_grid(resamples = heart_disease_vfold, grid = k_lots)



heart_disease_metrics_new <- knn_fit_new %>% 
                        collect_metrics()

accuracies_new <- heart_disease_metrics_new %>% 
        filter(.metric == "accuracy") 
accuracies_new
    
accuracy_vs_k_new <- ggplot(accuracies_new, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate") +
  theme(text = element_text(size = 12))

accuracy_vs_k_new

accuracies_final_new <- heart_disease_metrics_new %>% 
        filter(.metric == "accuracy") %>% 
        arrange(mean) %>% 
        tail(1)

accuracies_final_new




In [ ]:
knn_spec_2 <- nearest_neighbor(weight_func = "rectangular", neighbors = 23) %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

knn_fit_new <- workflow() %>% 
  add_recipe(heart_recipe_new) %>% 
  add_model(knn_spec_2) %>% 
  fit(data = heart_disease_train)

heart_prediction_new <- predict(knn_fit_new, heart_disease_test) %>% 
                    bind_cols(heart_disease_test)

heart_metrics_new <- heart_prediction_new %>% 
                metrics(truth = num, estimate = .pred_class) %>% 
                filter(.metric == "accuracy")

mnist_conf_mat_new <- heart_prediction_new %>% 
                 conf_mat(truth = num, estimate = .pred_class)
heart_metrics_new
mnist_conf_mat_new

### Questions to ask: 
- upsample --> change the predicted variable to binary categorical. 
- final visualization
- optimizing our number of k neighbours
- clean up R error
- is our classifier trained: validate with test data
- 
- do we need to compare our result with the initial hypothesis using graphs
- potential risks -- TODO


using five predictors: with high correlation.
binary categorical -- 0 and 1
final visulizatino with bar chart(quantitative and categorical), boxplot(quantitative): 

## Mutating Num to binary categorical

In [ ]:
heart_disease_data_3 <- read_csv(heart_disease_url, col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs",
                                                               "restecg", "thalach", "exang", "oldpeak", "slope",
                                                               "ca", "thal", "num")) %>% 
                    mutate(num = replace(num, num =='2',1)) %>% 
                    mutate(num = replace(num, num =='3',1)) %>% 
                    mutate(num = replace(num, num =='4',1)) %>% 
                    mutate(num = as.factor(num)) %>%  
                    select(cp, thalach, exang, slope, oldpeak, num) 
heart_disease_data_3    

## Check for balancing

In [ ]:
num_obs_new <- nrow(heart_disease_data_3)
        heart_disease_data_3 %>% 
        group_by(num) %>% 
        summarize(
        count = n(),
        percentage = n() / num_obs_new * 100)

## Spliting and training data

In [ ]:
set.seed(1234)
heart_disease_split_mut <- initial_split(heart_disease_data_3, prop = 0.75, strata = num)
heart_disease_train_mut <- training(heart_disease_split_mut)
heart_disease_test_mut <- testing(heart_disease_split_mut)

head(heart_disease_data_3, 10)

## Create model

In [ ]:
knn_spec_mut <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
            set_engine("kknn")%>%
            set_mode("classification") 
knn_spec_mut

In [ ]:
heart_recipe_mut <- recipe(num ~ ., data = heart_disease_train_mut)%>%
        step_scale(all_predictors())%>%
        step_center(all_predictors()) 

heart_disease_vfold_mut <- vfold_cv(heart_disease_train_mut, v = 5, strata = num)


In [ ]:
k_lots_mut <- tibble(neighbors = seq(from = 1, to = 50, by = 1))

knn_fit_mut <- workflow() %>% 
            add_recipe(heart_recipe_mut) %>% 
            add_model(knn_spec_mut) %>% 
            tune_grid(resamples = heart_disease_vfold_mut, grid = k_lots_mut) 

In [ ]:
heart_disease_metrics_mut <- knn_fit_mut %>% 
                        collect_metrics()

accuracies_mut <- heart_disease_metrics_mut %>% 
        filter(.metric == "accuracy") 
accuracies_mut
    
# accuracy_vs_k_mut <- ggplot(accuracies_mut, aes(x = neighbors, y = mean)) +
#   geom_point() +
#   geom_line() +
#   labs(x = "Neighbors", y = "Accuracy Estimate") +
#   theme(text = element_text(size = 12))

# accuracy_vs_k_mut

# accuracies_final_mut <- heart_disease_metrics_mut %>% 
#         filter(.metric == "accuracy") %>% 
#         arrange(mean) %>% 
#         tail(1)
# accuracies_final_mut

In [ ]:
accuracy_vs_k_mut <- ggplot(accuracies_mut, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate") +
  theme(text = element_text(size = 12))

accuracy_vs_k_mut

In [ ]:
accuracies_final_mut <- heart_disease_metrics_mut %>% 
        filter(.metric == "accuracy") %>% 
        arrange(mean) %>% 
        tail(1)
accuracies_final_mut

In [ ]:
knn_spec_mut <- nearest_neighbor(weight_func = "rectangular", neighbors = 26) %>% 
  set_engine("kknn") %>% 
  set_mode("classification")
knn_spec_mut

knn_fit_mut <- workflow() %>% 
  add_recipe(heart_recipe_mut) %>% 
  add_model(knn_spec_mut) %>% 
  fit(data = heart_disease_train_mut)
knn_fit_mut

In [ ]:
heart_prediction_mut <- predict(knn_fit_mut, heart_disease_test_mut) %>% 
                    bind_cols(heart_disease_test_mut)

heart_metrics_mut <- heart_prediction_mut %>% 
                metrics(truth = num, estimate = .pred_class) %>% 
                filter(.metric == "accuracy")

mnist_conf_mat_mut <- heart_prediction_mut %>% 
                 conf_mat(truth = num, estimate = .pred_class)

heart_prediction_mut
heart_metrics_mut
mnist_conf_mat_mut

### FINAL Visualization

In [ ]:
pred_vs_actual <- heart_prediction_mut %>% 
                select(.pred_class, cp, num) %>% 
                
                

In [ ]:
num_of_false_negative<-select(heart_prediction_mut, .pred_class == 1)

## 4 predictors

In [ ]:
heart_disease_data_4 <- read_csv(heart_disease_url, col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs",
                                                               "restecg", "thalach", "exang", "oldpeak", "slope",
                                                               "ca", "thal", "num")) %>% 
                    mutate(num = replace(num, num =='2',1)) %>% 
                    mutate(num = replace(num, num =='3',1)) %>% 
                    mutate(num = replace(num, num =='4',1)) %>% 
                    mutate(num = as.factor(num), cp = as.factor(cp), 
                          exang = as.factor(exang)) %>% 
                    select(cp, thalach, exang, oldpeak, num) 
heart_disease_data_4   

#  mutate(cp = as.factor(cp), fbs = as.factor(fbs), restecg = as.factor(restecg), 
#                             exang = as.factor(exang), slope = as.factor(slope), sex = as.factor(sex)) %>%

In [ ]:
set.seed(1234)
heart_disease_split_mut_4 <- initial_split(heart_disease_data_4, prop = 0.75, strata = num)
heart_disease_train_mut_4 <- training(heart_disease_split_mut_4)
heart_disease_test_mut_4 <- testing(heart_disease_split_mut_4)

head(heart_disease_data_4, 10)

In [ ]:
knn_spec_mut_4 <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
            set_engine("kknn")%>%
            set_mode("classification") 
knn_spec_mut_4

In [ ]:
heart_recipe_mut_new_4 <- recipe(num ~ ., data = heart_disease_train_mut_4)%>%
        step_scale(all_predictors())%>%
        step_center(all_predictors()) 

heart_disease_vfold_mut_4 <- vfold_cv(heart_disease_train_mut_4, v = 5, strata = num)

In [ ]:
k_lots_mut_4 <- tibble(neighbors = seq(from = 1, to = 80, by = 5))

knn_fit_mut_4 <- workflow() %>% 
            add_recipe(heart_recipe_mut_new_4) %>% 
            add_model(knn_spec_mut_4) %>% 
            tune_grid(resamples = heart_disease_vfold_mut_4, grid = k_lots_mut_4)

In [ ]:
heart_disease_metrics_mut_4 <- knn_fit_mut_4 %>% 
                        collect_metrics()

accuracies_mut_4 <- heart_disease_metrics_mut_4 %>% 
        filter(.metric == "accuracy") 
accuracies_mut_4
    
accuracy_vs_k_mut_4 <- ggplot(accuracies_mut_4, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate") +
  theme(text = element_text(size = 12))

accuracy_vs_k_mut_4

accuracies_final_mut_4 <- heart_disease_metrics_mut_4 %>% 
        filter(.metric == "accuracy") %>% 
        arrange(mean) %>% 
        tail(1)
accuracies_final_mut_4

In [ ]:
source("cleanup.R")